# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [73]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [74]:
# Output File (CSV)
output_data_file = "output_data/weather_df.csv"

In [75]:

weather2_df= pd.read_csv(output_data_file)

In [76]:
weather2_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,aden,99,YE,2020-10-22 16:07:26,61,12.78,45.04,85.77,2.39
1,port-gentil,94,GA,2020-10-22 16:07:26,85,-0.72,8.78,79.03,15.08
2,rikitea,100,PF,2020-10-22 16:06:33,77,-23.12,-134.97,73.90,16.35
3,souillac,20,MU,2020-10-22 16:07:27,73,-20.52,57.52,75.20,9.17
4,hobart,90,AU,2020-10-22 16:07:27,87,-42.88,147.33,53.60,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [77]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)


In [78]:
# Set the Latitud, Longitud and Humidity as parameters locations
locations = weather2_df[["Lat", "Lng"]].astype(float)
weights = weather2_df['Humidity'].astype(float)

In [85]:
# Create a Headmap base on our CSV file converted to Dataframe
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
# I considered narrow down the cities base on my ideal weather

# I conside a really goo temperature beween 64 and 75.
filter_1 = weather2_df.loc[weather2_df["Max Temp"]<76,:]
filter_2 = filter_1.loc[weather2_df["Max Temp"]>63,:]
filter_2.head()

# I prefered wind speed no more than 15 mph.
filter_3 = filter_2.loc[filter_2["Wind Speed"]<16,:]
filter_3.head()

# I prefered a few clouds.
filter_4 = filter_3.loc[filter_3["Cloudiness"]<5,:]
filter_4.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
21,faya,0,SA,2020-10-22 16:07:31,13,18.39,42.45,75.20,10.29
38,bredasdorp,0,ZA,2020-10-22 16:07:35,68,-34.53,20.04,68.00,12.75
43,port elizabeth,0,ZA,2020-10-22 16:07:36,78,-33.92,25.57,71.60,13.87
98,farafangana,0,MG,2020-10-22 16:07:49,83,-22.82,47.83,73.72,10.76
124,rawson,0,AR,2020-10-22 16:07:55,28,-43.30,-65.10,72.01,11.12


In [42]:
# Validate null value in the Dataframe
filter_4.isnull().values.any()

False

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [59]:
# Store my filter preference into hotel_df
hotel_df = filter_4.copy()


In [62]:
hotel_df.head()

,City,Cloudiness,Country,Hotel Name,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
21,faya,0,SA,True,2020-10-22 16:07:31,13,18.39,42.45,75.20,10.29
38,bredasdorp,0,ZA,True,2020-10-22 16:07:35,68,-34.53,20.04,68.00,12.75
43,port elizabeth,0,ZA,True,2020-10-22 16:07:36,78,-33.92,25.57,71.60,13.87
98,farafangana,0,MG,True,2020-10-22 16:07:49,83,-22.82,47.83,73.72,10.76
124,rawson,0,AR,True,2020-10-22 16:07:55,28,-43.30,-65.10,72.01,11.12


In [57]:
# Adding a Hotel Name column into my DF
hotel_df['Hotel Name']=""
hotel_df.head()

,City,Cloudiness,Country,Hotel Name,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
21,faya,0,SA,,2020-10-22 16:07:31,13,18.39,42.45,75.20,10.29
38,bredasdorp,0,ZA,,2020-10-22 16:07:35,68,-34.53,20.04,68.00,12.75
43,port elizabeth,0,ZA,,2020-10-22 16:07:36,78,-33.92,25.57,71.60,13.87
98,farafangana,0,MG,,2020-10-22 16:07:49,83,-22.82,47.83,73.72,10.76
124,rawson,0,AR,,2020-10-22 16:07:55,28,-43.30,-65.10,72.01,11.12


In [63]:
# Set parameters to search for hotels with 5000 meters base on my hotel_df
for index, row in hotel_df.iterrows():

    # Set variable with value of the Dataframe
    Lat = row['Lat']
    Lng = row['Lng']
    City = row['City']
  
    # create request to google
    print(f"Retrieving Results for Id City {index}: {City}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={Lat},{Lng}&radius=5000&type=hotel&key={g_key}").json()
    
    # Store the specific part of the json response named 'results'
    results = response['results']
    
    # Evaluate the Closest hotel base on Lat and Lng of each City
    try:
        hotel_name = response['results'][7]['name']
        print(f"Closest hotel to {City} at {Lat} , {Lng} is {hotel_name}.")
        
        # Store the hotel name closest into the Dataframe
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        
    except (KeyError, IndexError):
        print("Missing result... skipping...")
        
    print("------------------------------------------------------------------------------------")

Retrieving Results for Id City 21: faya.
Closest hotel to faya at 18.39 , 42.45 is مدرسة المعتصم بالله.
------------
Retrieving Results for Id City 38: bredasdorp.
Closest hotel to bredasdorp at -34.53 , 20.04 is Holiday Home Bloekomhuisie.
------------
Retrieving Results for Id City 43: port elizabeth.
Closest hotel to port elizabeth at -33.92 , 25.57 is Bhotani Lodge.
------------
Retrieving Results for Id City 98: farafangana.
Closest hotel to farafangana at -22.82 , 47.83 is TOHANAINA.
------------
Retrieving Results for Id City 124: rawson.
Closest hotel to rawson at -43.3 , -65.1 is Professional College of Engineering and Surveying.
------------
Retrieving Results for Id City 130: bac giang.
Closest hotel to bac giang at 21.27 , 106.2 is Private Hospital Song Thuong.
------------
Retrieving Results for Id City 146: hermanus.
Closest hotel to hermanus at -34.42 , 19.23 is Seaside Hermanus Guest Room.
------------
Retrieving Results for Id City 165: cap malheureux.
Closest hotel to

In [64]:
hotel_df.head()

,City,Cloudiness,Country,Hotel Name,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
21,faya,0,SA,مدرسة المعتصم بالله,2020-10-22 16:07:31,13,18.39,42.45,75.20,10.29
38,bredasdorp,0,ZA,Holiday Home Bloekomhuisie,2020-10-22 16:07:35,68,-34.53,20.04,68.00,12.75
43,port elizabeth,0,ZA,Bhotani Lodge,2020-10-22 16:07:36,78,-33.92,25.57,71.60,13.87
98,farafangana,0,MG,TOHANAINA,2020-10-22 16:07:49,83,-22.82,47.83,73.72,10.76
124,rawson,0,AR,Professional College of Engineering and Surveying,2020-10-22 16:07:55,28,-43.30,-65.10,72.01,11.12


In [82]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [101]:
weights = hotel_df['Humidity'].astype(float)

# Setting the gmap figure parameters
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

# Create the heatmap
heat_layer = gmaps.heatmap_layer(locations, weights=weights,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

# Create the markers into the gmap base on "hotel info"
marker_layer = gmaps.marker_layer(locations, hover_text='', label='', info_box_content=hotel_info)

# Add marker layer on the map
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display Map
fig





Figure(layout=FigureLayout(height='420px'))